In [1]:
import datamigration_wrapper as dms
from datetime import datetime
import pandas as pd
import multiprocessing
from multiprocessing import log_to_stderr, get_logger
import logging
import argparse
from pgdb import connect

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--migration-file', type=str)
args = parser.parse_args("")

migration_file = args.migration_file or 'Test GCP Migration.csv'
df_machine_types = pd.read_csv('machine_types.csv')
df_migration_data = pd.read_csv(migration_file)

In [3]:
display(df_migration_data)

,Team Name,Service,k8sSecretNamespace,DB Identifier,Endpoint Address,Port,ReplicationUsername,ReplicationPassword,ProjectId,Version,Location,StorageType,CPU,Memory,Storage
0,accounts-identity,fake,tmc-iam,poc-migration-12,poc-migration-12.cjq8ffbfbj4k.us-east-1.rds.am...,5432,postgres,postgres,aws-rds-gcp-cloudsql,POSTGRES_12,us-east4,PD_SSD,2,1,15
1,accounts-identity,fake,tmc-iam,poc-migration-11,poc-migration-11.cjq8ffbfbj4k.us-east-1.rds.am...,5432,postgres,postgres,aws-rds-gcp-cloudsql,POSTGRES_11,us-east4,PD_SSD,1,1,10
2,accounts-identity,fake,tmc-iam,poc-migration-12,poc-migration-12.cjq8ffbfbj4k.us-east-1.rds.am...,5432,postgres,postgres,aws-rds-gcp-cloudsql,POSTGRES_12,us-east4,PD_SSD,2,1,12
3,accounts-identity,fake,tmc-iam,poc-migration-11,poc-migration-11.cjq8ffbfbj4k.us-east-1.rds.am...,5432,postgres,postgres,aws-rds-gcp-cloudsql,POSTGRES_11,us-east4,PD_SSD,1,1,18


In [4]:
project_id = 'aws-rds-gcp-cloudsql'
region_id = 'us-east4'
prefix_cp_source = 'auto-cp-pg-'
prefix_cp_cloudsql = 'auto-cs-pg-'
prefix_mj='auto-mj-'
now_str = datetime.now().strftime("%Y%m%dt%H%M%S")

In [5]:
list_migration_job = []
for index, row in df_migration_data.iterrows():
    dc={}
    dc["source_connection"] = {"postgresql": {"host": row["Endpoint Address"],
                                      "port": row["Port"],
                                      "username": row["ReplicationUsername"],
                                      "password": row["ReplicationPassword"]}}
    dc["target_server_settings_cloud_sql"] = { "databaseVersion": row["Version"],
                              "tier": "db-custom-1-3840",
                              "dataDiskSizeGb": row["Storage"]}
    dc["location_dict"] = {"project_id": project_id,
                  "region_id": region_id}
    dc["prefix_dict"] = {"prefix_cp_source": prefix_cp_source,
               "prefix_cp_cloudsql": prefix_cp_cloudsql,
               "prefix_mj": prefix_mj,
               "now_str": now_str,
                "id" : index}
    dc["target_base_settings_cloud_sql"] = { "ipConfig": {
                                "enableIpv4": True
                              },
                              "autoStorageIncrease": True,
                              "dataDiskType": "PD_SSD",
                              "rootPassword": "postgres"}
    list_migration_job.append((dc["prefix_dict"],dc["location_dict"],dc["source_connection"],dc["target_base_settings_cloud_sql"],dc["target_server_settings_cloud_sql"]))

In [6]:
list_migration_job[0][2]

{'postgresql': {'host': 'poc-migration-12.cjq8ffbfbj4k.us-east-1.rds.amazonaws.com',
  'port': 5432,
  'username': 'postgres',
  'password': 'postgres'}}

In [7]:
source_connection = list_migration_job[0][2]#["postgresql"]

In [8]:
logger = dms.setup_logger()

In [9]:
dms.test_connection(source_connection,logger)

2021/05/11 23:52:51:root:DEBUG:Your connection to poc-migration-12.cjq8ffbfbj4k.us-east-1.rds.amazonaws.com was successful.


True

In [14]:
source_connection = dict(source_connection["postgresql"])
source_connection["database"]="postgres"
source_connection

{'host': 'poc-migration-12.cjq8ffbfbj4k.us-east-1.rds.amazonaws.com',
 'port': 5432,
 'username': 'postgres',
 'password': 'postgres',
 'database': 'postgres'}

In [15]:
conn = connect(database=source_connection['database'],
                         host=source_connection['host'],
                         user=source_connection['username'],
                         password=source_connection['password'],
                         port=source_connection['port'])

In [16]:
conn.close()

In [ ]:

try:
    conn = connect(database=source_connection['database'],
                         host=source_connection['host'],
                         user=source_connection['username'],
                         password=source_connection['password'],
                         port=source_connection['port'])
    conn.close()
    logger.debug("Your connection to {} was successful.".format(source_connection['host']))
    return True
except:
    logger.debug("Your connection to {} was not successful.".format(source_connection['host']))
    return False

In [17]:
logger.debug("Your connection to {} was successful.".format(source_connection['host']))

In [10]:
source_connection

{'host': 'poc-migration-12.cjq8ffbfbj4k.us-east-1.rds.amazonaws.com',
 'port': 5432,
 'username': 'postgres',
 'password': 'postgres'}

In [16]:
def test_connection(source_connection):
    source_connection = source_connection[:]
    source_connection["database"]="postgres"
    try:
        conn = connect(database=source_connection['database'],
                             host=source_connection['host'],
                             user=source_connection['username'],
                             password=source_connection['password'],
                             port=source_connection['port'])
        conn.close()
        return True
    except:
        return False

SyntaxError: invalid syntax (<ipython-input-16-5de3e2c51b5e>, line 1)